In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from sklearn.metrics import roc_curve
import statsmodels.api as sm
font_name = fm.FontProperties(fname='C:\Windows/Fonts/malgun.ttf').get_name()
plt.rc('font', family=font_name)
mpl.rcParams["axes.unicode_minus"] = False

import pymysql, math, time
from sklearn import preprocessing
import urllib

from IPython.display import Image

In [2]:
change_bright = pd.read_excel("data/퍼스널컬러 정량화 모델 가설검정을 위한 설문 (1).xlsx",
                          sheet_name = 'Sheet1', header=0,na_values = 'NaN',
                          usecols="C, D, E, F, G, H, S, T, U, V, W, X")
change_bright.rename(columns={change_bright.columns[0]:"sex",
                        change_bright.columns[1]:"age_class",
                        change_bright.columns[2]:"definition",
                        change_bright.columns[3]:"need",
                        change_bright.columns[4]:"purpose",
                        change_bright.columns[5]:"interest",
                        change_bright.columns[6]:"comp",
                        change_bright.columns[7]:"triangle1",
                        change_bright.columns[8]:"triangle2",
                        change_bright.columns[9]:"doubleness1",
                        change_bright.columns[10]:"doubleness2",
                        change_bright.columns[11]:"doubleness3"}, inplace=True)

In [3]:
for i in range(int(change_bright.shape[0])):
    # sex : Male, Female 리스트 담기
    if change_bright["sex"][i] == "남성":
        change_bright["sex"][i] = "Male"
    else:
        change_bright["sex"][i] = "Female"
        
    # age_class : 10, 20, 30, 40, 50으로 리스트 담기
    if change_bright["age_class"][i] == "10대":
        change_bright["age_class"][i] = "10"
    elif change_bright["age_class"][i] == "20대":
        change_bright["age_class"][i] = "20"
    elif change_bright["age_class"][i] == "30대":
        change_bright["age_class"][i] = "30"
    elif change_bright["age_class"][i] == "40대":
        change_bright["age_class"][i] = "40"
    else:
        change_bright["age_class"][i] = "50"
    
    # definition : bright, harmony로 리스트 담기
    if ("생기" in change_bright["definition"][i]):
        change_bright["definition"][i] = "bright"
    else:
        change_bright["definition"][i] = "harmony"
    
    # need :  bright, harmony로 리스트 담기
    if ("생기" in change_bright["need"][i]):
        change_bright["need"][i] = "bright"
    else:
        change_bright["need"][i] = "harmony"
    
    # purpose : clothes, cosmetic, etc로 리스트 담기
    if change_bright["purpose"][i] == "옷을 구매하는 경우에 활용":
        change_bright["purpose"][i] = "clothes"
    elif change_bright["purpose"][i] == "화장하는 경우에 활용":
        change_bright["purpose"][i] = "cosmetic"
    else:
        change_bright["purpose"][i] = "etc"

    # columns :1, 2, 3, 4 ,5 - 72, 144, origin, 216, 288
columns = ["comp", "triangle1", "triangle2", "doubleness1", "doubleness2", "doubleness3"]
for i in range(int(change_bright.shape[0])):
    for col in columns:
            if change_bright[col][i] == "①":
                change_bright[col][i] = "1"
            elif change_bright[col][i] == "②":
                change_bright[col][i] = "2"
            elif change_bright[col][i] == "③":
                change_bright[col][i] = "3"
            elif change_bright[col][i] == "④":
                change_bright[col][i] = "4"
            else:
                change_bright[col][i] = "5"

C:\Users\utan_\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\utan_\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\utan_\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\utan_\anaconda3\lib\site-pac

In [4]:
change_bright

,sex,age_class,definition,need,purpose,interest,comp,triangle1,triangle2,doubleness1,doubleness2,doubleness3
0,Male,20,harmony,harmony,clothes,5,4,5,2,3,2,2
1,Female,20,bright,harmony,clothes,5,1,1,2,2,3,2
2,Female,20,bright,bright,clothes,3,2,4,1,2,1,2
3,Female,20,harmony,harmony,clothes,3,5,5,4,5,4,1
4,Female,20,bright,bright,etc,4,4,2,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
112,Female,30,harmony,harmony,cosmetic,4,4,2,2,4,1,3
113,Male,30,harmony,bright,cosmetic,2,1,4,5,2,4,3
114,Female,20,bright,harmony,etc,4,4,5,4,1,2,5
115,Male,20,bright,harmony,clothes,2,3,2,5,4,5,5


In [5]:
# 기본항목 전처리 결과 저장
change_bright.to_csv("data/change_bright.csv")

In [6]:
# 불러오기
df = pd.read_csv("data/change_bright.csv")

In [7]:
df

,Unnamed: 0,sex,age_class,definition,need,purpose,interest,comp,triangle1,triangle2,doubleness1,doubleness2,doubleness3
0,0,Male,20,harmony,harmony,clothes,5,4,5,2,3,2,2
1,1,Female,20,bright,harmony,clothes,5,1,1,2,2,3,2
2,2,Female,20,bright,bright,clothes,3,2,4,1,2,1,2
3,3,Female,20,harmony,harmony,clothes,3,5,5,4,5,4,1
4,4,Female,20,bright,bright,etc,4,4,2,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,112,Female,30,harmony,harmony,cosmetic,4,4,2,2,4,1,3
113,113,Male,30,harmony,bright,cosmetic,2,1,4,5,2,4,3
114,114,Female,20,bright,harmony,etc,4,4,5,4,1,2,5
115,115,Male,20,bright,harmony,clothes,2,3,2,5,4,5,5


### 답과 관련된 변수 찾기

In [8]:
# 전처리가 필요없는 컬럼
cols_to_keep = ['comp', 'triangle1', 'triangle2','doubleness1','doubleness2','doubleness3']

keep_comp = ['comp', 'interest']

In [9]:
# comp항목에서 1번을 투표한 사람들을 1, 나머지를  0

for i in range(int(df.shape[0])):
    # sex : Male, Female 리스트 담기
    if df["comp"][i] == 2:
        df["comp"][i] = 1
    else:
        df["comp"][i] = 0

C:\Users\utan_\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\utan_\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
# interest 항목에서 1~3을 투표한사람들의 값을 1, 4~5에 투표한사람들을 2

for i in range(int(df.shape[0])):
    # sex : Male, Female 리스트 담기
    if df["interest"][i] == 1:
        df["interest"][i] = 1
    elif df["interest"][i] == 2:
        df["interest"][i] = 1
    elif df["interest"][i] == 3:
        df["interest"][i] = 1
    else:
        df["interest"][i] = 2

C:\Users\utan_\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\utan_\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\utan_\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\utan_\anaco

In [11]:
df

,Unnamed: 0,sex,age_class,definition,need,purpose,interest,comp,triangle1,triangle2,doubleness1,doubleness2,doubleness3
0,0,Male,20,harmony,harmony,clothes,2,0,5,2,3,2,2
1,1,Female,20,bright,harmony,clothes,2,0,1,2,2,3,2
2,2,Female,20,bright,bright,clothes,1,1,4,1,2,1,2
3,3,Female,20,harmony,harmony,clothes,1,0,5,4,5,4,1
4,4,Female,20,bright,bright,etc,2,0,2,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,112,Female,30,harmony,harmony,cosmetic,2,0,2,2,4,1,3
113,113,Male,30,harmony,bright,cosmetic,1,0,4,5,2,4,3
114,114,Female,20,bright,harmony,etc,2,0,5,4,1,2,5
115,115,Male,20,bright,harmony,clothes,1,0,2,5,4,5,5


In [12]:
dummy_sex = pd.get_dummies(df['sex'], prefix='sex')
dummy_sex.head()

,sex_Female,sex_Male
0,0,1
1,1,0
2,1,0
3,1,0
4,1,0


In [13]:
dummy_def = pd.get_dummies(df['definition'], prefix='definition')
dummy_def.head()

,definition_bright,definition_harmony
0,0,1
1,1,0
2,1,0
3,0,1
4,1,0


In [14]:
dummy_need = pd.get_dummies(df['need'], prefix='need')
dummy_need.head()

,need_bright,need_harmony
0,0,1
1,0,1
2,1,0
3,0,1
4,1,0


In [15]:
dummy_pur = pd.get_dummies(df['purpose'], prefix='purpose')
dummy_pur.head()

,purpose_clothes,purpose_cosmetic,purpose_etc
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,0,0,1


### 셀병합

In [16]:
df1 = df[keep_comp].join(dummy_sex.loc[:, 'sex_Male':])
df1.head()

,comp,interest,sex_Male
0,0,2,1
1,0,2,0
2,1,1,0
3,0,1,0
4,0,2,0


In [17]:
df1 = df1.join(dummy_def.loc[:, 'definition_harmony'])
df1.head()

,comp,interest,sex_Male,definition_harmony
0,0,2,1,1
1,0,2,0,0
2,1,1,0,0
3,0,1,0,1
4,0,2,0,0


In [18]:
# df1 = df1.join(dummy_need.loc[:, 'need_harmony'])
# df1.head()

In [19]:
df1 = df1.join(dummy_pur.loc[:, 'purpose':])
df1.head()

,comp,interest,sex_Male,definition_harmony,purpose_clothes,purpose_cosmetic,purpose_etc
0,0,2,1,1,1,0,0
1,0,2,0,0,1,0,0
2,1,1,0,0,1,0,0
3,0,1,0,1,1,0,0
4,0,2,0,0,0,0,1


In [20]:
# comp를 제외한 셀 추출
train_cols = df1.columns[1:]
train_cols

Index(['interest', 'sex_Male', 'definition_harmony', 'purpose_clothes',
       'purpose_cosmetic', 'purpose_etc'],
      dtype='object')

In [21]:
# 결측치 확인
np.isnan(df1[train_cols])

,interest,sex_Male,definition_harmony,purpose_clothes,purpose_cosmetic,purpose_etc
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
112,False,False,False,False,False,False
113,False,False,False,False,False,False
114,False,False,False,False,False,False
115,False,False,False,False,False,False


In [22]:
df1

,comp,interest,sex_Male,definition_harmony,purpose_clothes,purpose_cosmetic,purpose_etc
0,0,2,1,1,1,0,0
1,0,2,0,0,1,0,0
2,1,1,0,0,1,0,0
3,0,1,0,1,1,0,0
4,0,2,0,0,0,0,1
...,...,...,...,...,...,...,...
112,0,2,0,1,0,1,0
113,0,1,1,1,0,1,0
114,0,2,0,0,0,0,1
115,0,1,1,0,1,0,0


In [23]:
dfy = df1.iloc[:, 0]
dfX = sm.add_constant(df1[train_cols])
model1 = sm.OLS(dfy, dfX)

print(model1.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                   comp   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.7782
Date:                Tue, 17 Mar 2020   Prob (F-statistic):              0.567
Time:                        00:03:00   Log-Likelihood:                -47.296
No. Observations:                 117   AIC:                             106.6
Df Residuals:                     111   BIC:                             123.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.1728      0

In [24]:
# comp 1~5냅두고 계산했을떄
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   comp   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.022
Method:                 Least Squares   F-statistic:                    0.5068
Date:                Tue, 17 Mar 2020   Prob (F-statistic):              0.771
Time:                        00:01:33   Log-Likelihood:                -216.97
No. Observations:                 117   AIC:                             445.9
Df Residuals:                     111   BIC:                             462.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  2.5434      0.432      5.894      0.000       1.688       3.399
interest              -0.3093      0.301     -1.027      0.307      -0.906       0.287
sex_Male               0.0545      0.416      0.131      0.896      -0.769       0.878
definition_harmony    -0.4278      0.349     -1.226      0.223      -1.120       0.264
purpose_clothes        0.8711      0.268      3.252      0.002       0.340       1.402
purpose_cosmetic       0.9305      0.308      3.018      0.003       0.319       1.542
purpose_etc            0.7419      0.260      2.848      0.005       0.226       1.258
==============================================================================
Omnibus:                       68.623   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               11.622
Skew:                           0.430   Prob(JB):                      0.00299
Kurtosis:                       1.717   Cond. No.                     3.03e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.56e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.


IndentationError: unexpected indent (<ipython-input-24-485319703cf7>, line 2)

In [ ]:
# comp 1만 정답일때
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   comp   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.214
Date:                Tue, 17 Mar 2020   Prob (F-statistic):              0.307
Time:                        00:01:58   Log-Likelihood:                -78.095
No. Observations:                 117   AIC:                             168.2
Df Residuals:                     111   BIC:                             184.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0246      0.132      0.187      0.852      -0.236       0.286
interest               0.1196      0.092      1.302      0.196      -0.062       0.302
sex_Male               0.0372      0.127      0.294      0.770      -0.214       0.289
definition_harmony     0.2119      0.107      1.990      0.049       0.001       0.423
purpose_clothes       -0.0465      0.082     -0.568      0.571      -0.208       0.116
purpose_cosmetic       0.0079      0.094      0.084      0.933      -0.179       0.194
purpose_etc            0.0631      0.079      0.795      0.429      -0.094       0.221
==============================================================================
Omnibus:                     1045.053   Durbin-Watson:                   2.043
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               16.207
Skew:                           0.477   Prob(JB):                     0.000302
Kurtosis:                       1.446   Cond. No.                     3.03e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.56e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.


In [ ]:
# comp 2만 정답일떄
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   comp   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.7782
Date:                Tue, 17 Mar 2020   Prob (F-statistic):              0.567
Time:                        00:03:00   Log-Likelihood:                -47.296
No. Observations:                 117   AIC:                             106.6
Df Residuals:                     111   BIC:                             123.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.1728      0.101      1.708      0.091      -0.028       0.373
interest              -0.0033      0.071     -0.047      0.962      -0.143       0.137
sex_Male              -0.1050      0.097     -1.078      0.284      -0.298       0.088
definition_harmony    -0.0681      0.082     -0.832      0.407      -0.230       0.094
purpose_clothes        0.1537      0.063      2.447      0.016       0.029       0.278
purpose_cosmetic       0.0235      0.072      0.325      0.746      -0.120       0.167
purpose_etc           -0.0044      0.061     -0.072      0.943      -0.125       0.117
==============================================================================
Omnibus:                       40.389   Durbin-Watson:                   2.058
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.087
Skew:                           1.740   Prob(JB):                     2.71e-15
Kurtosis:                       4.283   Cond. No.                     3.03e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.56e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
